In [2]:
# Import Pandas for Table Scraping
import pandas as pd

In [3]:
# Import Geopy and Pycountry for Lat/Long
from geopy.geocoders import Nominatim
import pycountry

In [4]:
# Read in HTML tables into a DataFrame
df = pd.read_html('https://en.wikipedia.org/wiki/Donald_Trump%27s_Liberation_Day_speech#List_of_tariffs_imposed.html')

In [7]:
# Create a DataFrame for the Tariff Table (Which is the fifth table on the page)
tariff_df = df[4]

In [8]:
# Verify we got the right table and that it has at least 100 data points
tariff_df

,Countries and Territories imposed upon,Percentage
0,Afghanistan,10%
1,Albania,10%
2,Algeria,30%
3,Andorra,10%
4,Angola,32%
...,...,...
180,Venezuela,15%
181,Vietnam,46%
182,Yemen,10%
183,Zambia,17%


In [9]:
# Rename the columns to make them easier to work with
tariff_df = tariff_df.rename(columns={'Percentage':"Tariff Percentage", "Countries and Territories imposed upon": "Country"})
tariff_df

,Country,Tariff Percentage
0,Afghanistan,10%
1,Albania,10%
2,Algeria,30%
3,Andorra,10%
4,Angola,32%
...,...,...
180,Venezuela,15%
181,Vietnam,46%
182,Yemen,10%
183,Zambia,17%


In [11]:
# Create a function to turn country names into latitude and longitude coordinates
def get_coordinates(country_name):
    try:
        country = pycountry.countries.get(name=country_name)
        geolocator = Nominatim(user_agent="geo_app", timeout=3) 
        location = geolocator.geocode(country.name)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except AttributeError:
        return None, None

In [12]:
# Get a list of the country names
countries = tariff_df["Country"].tolist()
print(countries)

['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Azerbaijan', 'The Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'British Indian Ocean Territory', 'British Virgin Islands', 'Brunei', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands', 'Colombia', 'Comoros', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Curaçao', 'Democratic Republic of the Congo', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Ethiopia', 'European Union', 'Falkland Islands', 'Fiji', 'French Guiana', 'French Polynesia', 'Gabon', 'The Gambia', 'Georgia', 'Ghana', 'Gibraltar', 'Grenada', 'Guadeloupe', 'Guatemala', 'Guinea-Bissau', 'Guinea',

In [13]:
# Create a dictionary to hold location information
locations = []

# Run the function on the new countries list
# Add each country with its location data to the empty locations list
for country in countries:
    lat, lon = get_coordinates(country)
    if lat and lon:
        location = {"Country": country, "Latitude": float(lat), "Longitude": float(lon)}
    else:
        location = {"Country": country, "Latitude": float(0), "Longitude": float(0)}
    locations.append(location)

In [14]:
# Turn the locations list into a DataFrame
location_df = pd.DataFrame(locations)
location_df

,Country,Latitude,Longitude
0,Afghanistan,33.768006,66.238514
1,Albania,5.758765,-73.915162
2,Algeria,28.000027,2.999983
3,Andorra,42.540717,1.573203
4,Angola,-11.877577,17.569124
...,...,...,...
180,Venezuela,0.000000,0.000000
181,Vietnam,0.000000,0.000000
182,Yemen,16.347124,47.891527
183,Zambia,-14.518912,27.558988


In [15]:
# Verify the each country was accounted for
location_df.count()

Country      185
Latitude     185
Longitude    185
dtype: int64

In [16]:
# Check the datatypes
location_df.dtypes

Country       object
Latitude     float64
Longitude    float64
dtype: object

In [17]:
# Merge dataframes
merged = pd.merge(location_df, tariff_df, on= "Country", how= "left")

merged

,Country,Latitude,Longitude,Tariff Percentage
0,Afghanistan,33.768006,66.238514,10%
1,Albania,5.758765,-73.915162,10%
2,Algeria,28.000027,2.999983,30%
3,Andorra,42.540717,1.573203,10%
4,Angola,-11.877577,17.569124,32%
...,...,...,...,...
180,Venezuela,0.000000,0.000000,15%
181,Vietnam,0.000000,0.000000,46%
182,Yemen,16.347124,47.891527,10%
183,Zambia,-14.518912,27.558988,17%


In [18]:
# Identify the countries with missing location data
missing = merged[merged["Longitude"] == 0]
missing

,Country,Latitude,Longitude,Tariff Percentage
12,The Bahamas,0.0,0.0,10%
20,Bolivia,0.0,0.0,10%
25,British Virgin Islands,0.0,0.0,10%
26,Brunei,0.0,0.0,24%
44,Democratic Republic of the Congo,0.0,0.0,11%
55,European Union,0.0,0.0,20%
56,Falkland Islands,0.0,0.0,41%
61,The Gambia,0.0,0.0,10%
72,Heard and McDonald Islands,0.0,0.0,10%
77,Iran,0.0,0.0,10%


In [19]:
# Drop the rows with mising data
cleaned_df = merged[merged["Longitude"] != 0]
cleaned_df

,Country,Latitude,Longitude,Tariff Percentage
0,Afghanistan,33.768006,66.238514,10%
1,Albania,5.758765,-73.915162,10%
2,Algeria,28.000027,2.999983,30%
3,Andorra,42.540717,1.573203,10%
4,Angola,-11.877577,17.569124,32%
...,...,...,...,...
178,Uzbekistan,41.323730,63.952810,10%
179,Vanuatu,-16.525507,168.106915,22%
182,Yemen,16.347124,47.891527,10%
183,Zambia,-14.518912,27.558988,17%


In [21]:
# Get rid of percent sign
cleaned_df['Tariff Percentage'] = cleaned_df['Tariff Percentage'].str.replace('%', '', regex=False)

cleaned_df

/var/folders/ss/srgrjk491n37xhqx74rcv20m0000gn/T/ipykernel_13508/11354116.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Tariff Percentage'] = cleaned_df['Tariff Percentage'].str.replace('%', '', regex=False)


,Country,Latitude,Longitude,Tariff Percentage
0,Afghanistan,33.768006,66.238514,10
1,Albania,5.758765,-73.915162,10
2,Algeria,28.000027,2.999983,30
3,Andorra,42.540717,1.573203,10
4,Angola,-11.877577,17.569124,32
...,...,...,...,...
178,Uzbekistan,41.323730,63.952810,10
179,Vanuatu,-16.525507,168.106915,22
182,Yemen,16.347124,47.891527,10
183,Zambia,-14.518912,27.558988,17


In [ ]:
# Change the variable type for Tariff Percentage to integer
cleaned_df['Tariff Percentage'] = cleaned_df['Tariff Percentage'].astype(int)
cleaned_df.dtypes

/var/folders/ss/srgrjk491n37xhqx74rcv20m0000gn/T/ipykernel_13508/378375694.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Tariff Percentage'] = cleaned_df['Tariff Percentage'].astype(int)


Country               object
Latitude             float64
Longitude            float64
Tariff Percentage      int64
dtype: object

In [25]:
cleaned_df

,Country,Latitude,Longitude,Tariff Percentage
0,Afghanistan,33.768006,66.238514,10
1,Albania,5.758765,-73.915162,10
2,Algeria,28.000027,2.999983,30
3,Andorra,42.540717,1.573203,10
4,Angola,-11.877577,17.569124,32
...,...,...,...,...
178,Uzbekistan,41.323730,63.952810,10
179,Vanuatu,-16.525507,168.106915,22
182,Yemen,16.347124,47.891527,10
183,Zambia,-14.518912,27.558988,17


In [ ]:
# Export to a json file for JavaScript
cleaned_df.to_json('Data.json')

In [ ]:
# Export to csv file for SQL
cleaned_df.to_csv('Data.csv', index=False)